In [2]:
import h5py
import numpy as np
from sklearn.utils import shuffle


# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

e:\conda\envs\python39_usual\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "
e:\conda\envs\python39_usual\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:

X_train = []
X_test = []

for filename in ['D:\data\kaggle\cat_dog\my_gap\gap_my_ResNet50.h5', 
                 'D:\data\kaggle\cat_dog\my_gap\gap_my_Xception.h5', 
                 'D:\data\kaggle\cat_dog\my_gap\gap_my_InceptionV3.h5']:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [4]:
from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
# 使用load_model函数加载模型

model = load_model(r"C:\Users\haokw\Documents\GitHub\VScode\cat_dog\model\model_my_best_3.h5")

# 查看模型的架构
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6144)]            0         
                                                                 
 dropout (Dropout)           (None, 6144)              0         
                                                                 
 dense (Dense)               (None, 1)                 6145      
                                                                 
Total params: 6,145
Trainable params: 6,145
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(X_train, y_train, batch_size=512, epochs=10000, validation_split=0.2)

Epoch 1/10000
40/40 [==============================] - 1s 26ms/step - loss: 0.0191 - accuracy: 0.9934 - val_loss: 0.0184 - val_accuracy: 0.9952
Epoch 2/10000
40/40 [==============================] - 1s 25ms/step - loss: 0.0186 - accuracy: 0.9941 - val_loss: 0.0184 - val_accuracy: 0.9952
Epoch 3/10000
40/40 [==============================] - 1s 23ms/step - loss: 0.0200 - accuracy: 0.9927 - val_loss: 0.0184 - val_accuracy: 0.9952
Epoch 4/10000
40/40 [==============================] - 1s 22ms/step - loss: 0.0192 - accuracy: 0.9935 - val_loss: 0.0184 - val_accuracy: 0.9952
Epoch 5/10000
40/40 [==============================] - 1s 24ms/step - loss: 0.0197 - accuracy: 0.9938 - val_loss: 0.0184 - val_accuracy: 0.9952
Epoch 6/10000
40/40 [==============================] - 1s 24ms/step - loss: 0.0191 - accuracy: 0.9941 - val_loss: 0.0184 - val_accuracy: 0.9952
Epoch 7/10000
40/40 [==============================] - 1s 22ms/step - loss: 0.0197 - accuracy: 0.9932 - val_loss: 0.0184 - val_accuracy:

: 

In [ ]:
model.save('model\\model_myyy_5.h5')

In [ ]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

In [ ]:
import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("result\pred.csv")
df2 = pd.read_csv("result\pred_name.csv")
image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(r"D:\data\kaggle\cat_dog\test", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    df.at[index-1, 'label'] = y_pred[i]
    if y_pred[i]>0.50:
        df2.at[index-1, 'label'] = "狗"
    else:
        df2.at[index-1, 'label'] = "猫"


df.to_csv('result\\pred.csv', index=None)
df.head(10)
df2.to_csv('result\\pred_name.csv', index=None)
df2.head(10)